In [1]:
import matplotlib.pyplot as plt
%matplotlib inline  
import numpy as np
from math import exp, log, pi
import scipy.stats as si
from scipy.integrate import quadrature,quad
import sympy as sy
from sympy.stats import Normal, cdf
from sympy import init_printing
init_printing()

def populate_y(x_values,function):
    y=[]
    for i in x_values:
        y.append(function(i))
    return y

#this function is to plot coustom function
def plot_function(function,start,end):
    #x_plt and y_plt are jut to p\lot Runge function 
    x_plt = np.arange(start, end, 0.001)   # start,stop,step
    y_plt=[]
    for j in x_plt:
        y_plt.append(function(j))
    return x_plt,y_plt

#Chebyshev points of the second kind
def generate_cheb_nodes(a,b,no_of_nodes):
    yp=[]
    n=no_of_nodes-1
    for i in range(no_of_nodes):
        yp.append((a+(0.5*(b-a)*((np.cos(i*np.pi/n))+1))))
    return yp

In [2]:
def chebyshev_coef_1d ( nd, xd, yd ):


#  Parameters:
#
#    Input, integer ND, the number of Chebyshev points.
#    ND must be at least 1.
#
#    Input, real XD(ND), the data locations.
#
#    Input, real YD(ND), the data values.
#
#    Output, real C(ND), the Chebyshev coefficients.
#
#    Output, real XMIN, XMAX, the interpolation interval.
#
    xmin = min ( xd )
    xmax = max ( xd )
#
#  Map XD to [-1,+1].
#
    x=[]
    for i in xd:
        x.append(( 2.0 * i - xmin - xmax ) / ( xmax - xmin ))
        
    n=nd-1  #n is degree 
    
    c=[]
    for j in range(nd):
        temp=0
        for k in range(nd):
            if k==0 or k==n:
                temp=temp+(.5*yd[k]*np.cos(j*np.arccos(x[k])))
            else:
                temp=temp+(yd[k]*np.cos(j*np.arccos(x[k])))               
        if j==0 or j==n:
            c.append((1/n)*temp)
        else:
            c.append((2/n)*temp)


    return c, xmin, xmax

def chebyshev_interp_1d ( nd, xd, yd, xi ):
    c, xmin, xmax = chebyshev_coef_1d ( nd, xd, yd )
    yi = chebyshev_value_1d ( nd, c, xmin, xmax,  xi )
    return yi

def chebyshev_value_1d ( nd, c, xmin, xmax, xi ):


## CHEBYSHEV_VALUE_1D evaluates a Chebyshev interpolant, given its coefficients.
#
#  Parameters:
#
#    Input, integer ND, the number of data points.
#    ND must be at least 1.
#
#    Input, real C(ND), the Chebyshev coefficients.
#
#    Output, real XMIN, XMAX, the interpolation interval.
#
#    Input, real XI, the interpolation points, which
#    must be each be in the interval [XMIN,XMAX].
#
#    Output, real YI, the interpolated values.
#

    if ( nd == 1 ):
        yi = c.copy ( )
        return yi
#
#  Map XI to [-1,+1].
#
    x = ( 2.0 * xi - xmin - xmax ) / ( xmax - xmin )
    yi=0
    for j in range(nd):
        yi=yi+(c[j]*np.cos(j*np.arccos(x)))
    return yi

In [3]:
#Testing Dynamic Chebyshev Method for European call price 

S0=50
K=100
T=1
r=.05
sigma=.25

#d_t function returns dt which is the t_u - t_(u-1)
def d_t(T,n):
    return (T/n)

#mean and std is calculated as mentioned in 4.2    Computation of generalised moments in dissertation
def mu_and_sigma(xmin,xmax,sigma,r,dt):

    u=1-(2*(xmax/(xmax-xmin)))+(2*((r-((sigma**2)/2))*dt)/(xmax-xmin)) #calculated as shown above in img
    sig=2*sigma*dt/(xmax-xmin) #calculated as shown above in img
    return u,sig

#this funtion computes the generalised moments matrix and expectation is calculated using integrating technique traditional quad
def gamma(N,nodes,u,sig,xmin,xmax):
    gam=np.zeros(shape=(N,N))
    for k in range(N):
        for j in range(N):
            def function(y): 
                return np.cos(j*np.arccos(y))*si.norm((u+(2*nodes[k]/(xmax-xmin))), sig).pdf(y)
            gam[k][j],_=quad(function, -1, 1)    
    return gam
       
def payoff_call(S0, x1,K):
    return max(exp(log(S0)+x1)-K, 0)

def payoff_put( S0, x1,K):
    return max(K-exp(log(S0)+x1), 0)

#this funtion is used in the iterative time stepping step in the DC algorithm for bermudan option
def V_tu(N,gam,c,nodes,payoffs,dt,r):

    V_t0=[]
    for k in range(N):
        temp2=0
        for j in range(N):
            temp2=temp2+(c[j]*gam[k][j])
        V_t0.append(max(payoffs[k],temp2*exp(-r*dt))) 
        
    return V_t0 

#this funtion is used in the iterative time stepping step in the DC algorithm for barrier option, the diffrent the previous if here the value funtion got payoff funtion as zero
def V_tu_b(N,gam,c,nodes,dt,r):

    V_t0=[]
    for k in range(N):
        temp2=0
        for j in range(N):
            temp2=temp2+(c[j]*gam[k][j])
        V_t0.append(temp2*exp(-r*dt))         
    return V_t0 

#this funtion evaluates the value at time step T and finds payoff at each Chebyshev nodes
def setup(xmin,xmax,N,payoff,S0,K):

    nodes=generate_cheb_nodes(xmin,xmax,N) #this contains all the x_k Chebyshev nodes
    payoffs=[]
    for i in nodes:
        payoffs.append(payoff(S0,i,K))
    c, _,_=chebyshev_coef_1d ( N, nodes, payoffs ) 
    return c,nodes,payoffs
#this funtion  calculates price at time step t=0 at Chebyshev nodes x=0 which is log(S_0/S_0)=0
def price(N,nodes,V_t0):
    return chebyshev_interp_1d ( N, nodes, V_t0, 0 )


In [4]:
def european_call(S0, K, T, r, sigma,N,xmin,xmax):
    dt=d_t(T,1)
    u,sig=mu_and_sigma(xmin,xmax,sigma,r,dt)
    c,nodes,payoffs=setup(xmin,xmax,N,payoff_call,S0,K)
    gam=gamma(N,nodes,u,sig,xmin,xmax)
    V_t0=[]
    for k in range(N):
        temp2=0
        for j in range(N):
            temp2=temp2+(c[j]*gam[k][j])
        V_t0.append(temp2*exp(-r*dt))        
    return price(N,nodes,V_t0)

def european_put(S0, K, T, r, sigma,N,xmin,xmax):
    dt=d_t(T,1)
    u,sig=mu_and_sigma(xmin,xmax,sigma,r,dt)
    c,nodes,payoffs=setup(xmin,xmax,N,payoff_put,S0,K)
    gam=gamma(N,nodes,u,sig,xmin,xmax)
    V_t0=[]
    for k in range(N):
        temp2=0
        for j in range(N):
            temp2=temp2+(c[j]*gam[k][j])
        V_t0.append(temp2*exp(-r*dt))        
    return price(N,nodes,V_t0)

def bermuda_call(S0, K, T, r, sigma,N,xmin,xmax,n):
    dt=d_t(T,n)
    u,sig=mu_and_sigma(xmin,xmax,sigma,r,dt)
    c,nodes,payoffs=setup(xmin,xmax,N,payoff_call,S0,K)
    gam=gamma(N,nodes,u,sig,xmin,xmax)
    for i in range(n):
        if i==0:
            vt=V_tu(N,gam,c,nodes,payoffs,dt,r)
            c1, _,_=chebyshev_coef_1d ( N, nodes, vt )
        elif i==n-1:
            vt=V_tu(N,gam,c1,nodes,payoffs,dt,r)
            price1=price(N,nodes,vt)
        else: 
            vt=V_tu(N,gam,c1,nodes,payoffs,dt,r)
            c1, _,_=chebyshev_coef_1d ( N, nodes, vt )
    return price1

def bermuda_put(S0, K, T, r, sigma,N,xmin,xmax,n):
    dt=d_t(T,n)
    u,sig=mu_and_sigma(xmin,xmax,sigma,r,dt)
    c,nodes,payoffs=setup(xmin,xmax,N,payoff_put,S0,K)
    gam=gamma(N,nodes,u,sig,xmin,xmax)
    for i in range(n):
        if i==0:
            vt=V_tu(N,gam,c,nodes,payoffs,dt,r)
            c1, _,_=chebyshev_coef_1d ( N, nodes, vt )
        elif i==n-1:
            vt=V_tu(N,gam,c1,nodes,payoffs,dt,r)
            price1=price(N,nodes,vt)
        else: 
            vt=V_tu(N,gam,c1,nodes,payoffs,dt,r)
            c1, _,_=chebyshev_coef_1d ( N, nodes, vt )
    return price1

def barrier_call(S0, K, T, r, sigma,N,monitoring_interval,B,barrier_type):
    n=monitoring_interval
    b=log(B/S0)
    dt=d_t(T,n)
    xmin,xmax=0,0
    if barrier_type=="up-and-out":
        xmin=-2.33
        xman=b
    elif barrier_type=="down-and-out":
        xmin=b
        xman=2
    elif barrier_type=="up-and-in":
        xmin=b
        xman=2
    elif barrier_type=="down-and-in":
        xmin=-2
        xman=b
    print(xmax,xmin)
    u,sig=mu_and_sigma(xmin,xmax,sigma,r,dt)
    c,nodes,payoffs=setup(xmin,xmax,N,payoff_call,S0,K)
    gam=gamma(N,nodes,u,sig,xmin,xmax)
    for i in range(n):
        if i==0:
            vt=V_tu_b(N,gam,c,nodes,dt,r)
            c1, _,_=chebyshev_coef_1d ( N, nodes, vt )
        elif i==n-1:
            vt=V_tu_b(N,gam,c1,nodes,dt,r)
            price1=price(N,nodes,vt)
        else: 
            vt=V_tu_b(N,gam,c1,nodes,dt,r)
            c1, _,_=chebyshev_coef_1d ( N, nodes, vt )
    return price1

def barrier_put(S0, K, T, r, sigma,N,monitoring_interval,b,barrier_type):
    n=monitoring_interval
    b=log(B/S0)
    dt=d_t(T,n)
    xmin,xmax=0,0
    if barrier_type=="up-and-out":
        xmin=-2
        xman=b
    elif barrier_type=="down-and-out":
        xmin=b
        xman=2
    elif barrier_type=="up-and-in":
        xmin=b
        xman=2
    elif barrier_type=="down-and-in":
        xmin=-2
        xman=b
    u,sig=mu_and_sigma(xmin,xmax,sigma,r,dt)
    c,nodes,payoffs=setup(xmin,xmax,N,payoff_put,S0,K)
    gam=gamma(N,nodes,u,sig,xmin,xmax)
    for i in range(n):
        if i==0:
            vt=V_tu_b(N,gam,c,nodes,payoffs,dt,r)
            c1, _,_=chebyshev_coef_1d ( N, nodes, vt )
        elif i==n-1:
            vt=V_tu_b(N,gam,c1,nodes,payoffs,dt,r)
            price1=price(N,nodes,vt)
        else: 
            vt=V_tu_b(N,gam,c1,nodes,payoffs,dt,r)
            c1, _,_=chebyshev_coef_1d ( N, nodes, vt )
    return price1

In [5]:
#analytic function for european call under Black-scholes model 
def euro_vanilla_call(S, K, T, r, sigma):
    
    #S: spot price
    #K: strike price
    #T: time to maturity
    #r: interest rate
    #sigma: volatility of underlying asset
    
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = (np.log(S / K) + (r - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    
    call = (S * si.norm.cdf(d1, 0.0, 1.0) - K * np.exp(-r * T) * si.norm.cdf(d2, 0.0, 1.0))
    
    return call

print("European call price for S=50, K=100, T=1, r=0.05, sigma=0.15 is ")
eurocall=euro_vanilla_call(100, 120, 1, 0.03, 0.15)
print(eurocall)

European call price for S=50, K=100, T=1, r=0.05, sigma=0.15 is 
1.307003119694862


In [10]:
xmin=-2.5
xmax=2.5
S0=100
K=120
T=1
r=.03
sigma=0.15
european_call(S0, K, T, r, sigma,100,xmin,xmax)


In [11]:
print("done")

done


### 